In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
import json

with open('funbigfile.json') as f:
    data = json.load(f)

df = pd.json_normalize(data['bruh'])
df = df.drop(['type', 'uri', 'track_href', 'analysis_url','time_signature', 'duration_ms'], axis=1)
#print(df)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('genre', axis=1), df['genre'], test_size=0.2, random_state=42)

# Encode the target variable
lb = LabelBinarizer()
y_train_encoded = lb.fit_transform(y_train)
y_test_encoded = lb.transform(y_test)

# Define the features to be used for training
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'valence']
#features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'speechiness', 'tempo', 'valence']

# Create the feature matrix
X_train = X_train[features]
X_test = X_test[features]

# Train a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train_encoded)

#debugging stuff:
print("Testing dataset:")
print(type(y_test))
#lowkey don't need this print(X_test)
print(y_test)
#more debugging stuff 
y_pred = rf.predict(X_test[features])
print("Predicted genres:")
print(type(lb.inverse_transform(y_pred)))
print(lb.inverse_transform(y_pred))


# Evaluate the performance of the model on the test data
accuracy = rf.score(X_test, y_test_encoded)
print("Accuracy:", accuracy)

#with open('smallfile.json') as g:
#    new_data = json.load(g)

#dff = pd.json_normalize(new_data['bruh'])  # Load new JSON data
#dff = dff.drop(['type', 'uri', 'track_href', 'analysis_url','time_signature', 'duration_ms'], axis=1)

Testing dataset:
<class 'pandas.core.series.Series'>
30     classicpiano
181            Rock
223             Pop
185            Rock
211             Pop
258             Pop
173            Rock
142            Rock
9      classicpiano
177            Rock
24     classicpiano
213             Pop
77     classicpiano
204             Pop
190            Rock
206             Pop
242             Pop
6      classicpiano
220             Pop
104            Rock
114            Rock
119            Rock
19     classicpiano
201             Pop
179            Rock
46     classicpiano
75     classicpiano
45     classicpiano
154            Rock
15     classicpiano
236             Pop
228             Pop
144            Rock
33     classicpiano
196            Rock
101            Rock
212             Pop
10     classicpiano
113            Rock
96     classicpiano
92     classicpiano
150            Rock
25     classicpiano
158            Rock
18     classicpiano
237             Pop
139            Rock
68     